# Quantum annealing - Benchmark: 

Les machines de quantum annealing (comme celles de D-Wave) prennent en entrée un problème sous forme QUBO. Pour cette raison, 
cherchons d'abord à écrire notre hamiltonien sous forme QUBU. \

Selon le résultat établi en [\ref{Equation: particulier}], on a en utilisant les variables binaires:

$H_C=\sum_{i,k}(c_i+\lambda-2\lambda D)x_{i,k}+\lambda \sum_{(i,k)\neq (j,l)} x_{i,k} x_{j,l}+\lambda D^2$

Posons: $a_{i,k}=c_i+2\lambda-2\lambda D$
 et $b_{(i,k),(j,l)}=\lambda$

Soit: $H_{QUBO}=\sum_{i,k}a_{i,k}x_{i,k}+\sum_{(i,k)\neq (j,l)} b_{(i,k),(j,l)}x_{i,k}x_{j,l}+ cte = x^TQx$

## Représentation naive de $g_i$

Dans le code normalement on découpe l'intervalle $[0, P_i]$ en petits pas de taille $\delta$. J'ai mis donc un cas général si on souhaite plus de précision au niveau des résultats. 
 
Si $P_i = 4$ et $\delta = 1$, alors $g_i$ peut prendre les valeurs  
$\{0, 1, 2, 3, 4\}$. Comme fait pour QAOA.

---- Représentation binaire ----


Pour chaque $g_i$, on utilise 
$k_i = \frac{P_i}{\delta}$
variables binaires.  

Chaque variable binaire $x_{i,j}$ représente une portion de $g_i$, et l’on a :
$g_i = \delta \cdot \sum_{j=0}^{k_i - 1} x_{i,j}$.
 
Si $P_i = 4$, $\delta = 1 \Rightarrow k_i = 4$ variables binaires.  
Si $(x_{i,0}, x_{i,1}, x_{i,2}, x_{i,3}) = (1,1,0,0)$, alors :
$g_i = 1 \cdot 1 + 1 \cdot 1 + 0 + 0 = 2$.

## 2 noeuds

In [10]:
import numpy as np
from dwave.samplers import SimulatedAnnealingSampler

# Paramètres du problème
P_i = [4, 2]       # Puissance max par noeud
c_i = [3, 8]        # Coût par unité de puissance
D_i = [3, 1]        # Demande par noeud
delta = 1           # Pas de discrétisation
lam = 1000          # Coefficient de pénalité (augmenté)

# la demande totale D
D = np.sum(D_i)

# Nombre de variables binaires par noeud
k = [int(p / delta) for p in P_i]  #[4,2]
total_vars = sum(k)                 # 6 variables binaires

# Mapping des variables (i, j) -> index global
mapping = {}
index = 0
for i in range(len(P_i)):
    for j in range(k[i]):
        mapping[(i, j)] = index
        index += 1

# Construction du QUBO
Q = {}
for i in range(len(P_i)):
    for j in range(k[i]):
        u = mapping[(i, j)]
        # Terme linéaire: coût + pénalité
        Q[(u, u)] = c_i[i] * delta + lam * (delta**2 - 2 * delta * D)
        
        # Termes quadratiques (paires de variables)
        for v in range(u + 1, total_vars):
            Q[(u, v)] = 2 * lam * (delta**2)

# Résolution avec quantum annealing
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample_qubo(Q, num_reads=1000)
sample = sampleset.first.sample

#Reconstruction des g_i
g = [0] * len(P_i)
for i in range(len(P_i)):
    for j in range(k[i]):
        if sample[mapping[(i, j)]] == 1:
            g[i] += delta

# Vérification des résultats
print(f"Solution optimale: g = {g}")
print(f"Coût total: {sum(c_i[i] * g[i] for i in range(len(g)))}")
print(f"Demande totale: {sum(g)} (cible: {D})")

Solution optimale: g = [4, 0]
Coût total: 12
Demande totale: 4 (cible: 4)


## Résolution classique

In [54]:
import gurobipy as gp
from gurobipy import GRB


noeuds = [0, 1, 2, 3]
P_i = [15, 20, 10, 25]  # Puissances max (MW)
c_i = [40, 35, 50, 30]  # Coûts (€/MWh)
D_i = [12, 8, 5, 15]    # Demandes (MW)
D_total = sum(D_i)       # Demande totale (40 MW)

model = gp.Model("OptimisationProduction")
model.setParam('OutputFlag', 0) # pour n'afficher que le résultat final (sans détails)

# Variables de décision
g = model.addVars(noeuds, lb=0.0, vtype=GRB.CONTINUOUS, name="production")

#Fonction objectif (minimiser les coûts)
model.setObjective(gp.quicksum(c_i[i] * g[i] for i in noeuds), GRB.MINIMIZE)

#Contraintes de capacité
capacity_constrs = model.addConstrs(
    (g[i] <= P_i[i] for i in noeuds), 
    name="capacity"
)

#Contrainte d'équilibre offre-demande
balance_constr = model.addConstr(
    gp.quicksum(g[i] for i in noeuds) == D_total, 
    name="balance"
)

model.optimize()
if model.status == GRB.OPTIMAL:
    print("\nSOLUTION OPTIMALE:")
    print(f"Coût total: {model.objVal:.2f} €")
    print(f"Demande totale: {D_total} MW")
    
    total_prod = 0
    for i in noeuds:
        prod_val = g[i].X
        total_prod += prod_val
        print(f"Nœud {i+1}: {prod_val:.1f}/{P_i[i]} MW " f"(coût: {c_i[i]} €/MWh)")
    
    print(f"\nProduction totale: {total_prod} MW")
    print("Vérification contraintes:")
    print(f"- Équilibre offre/demande: {'OK' if abs(total_prod - D_total) < 1e-6 else 'ERREUR'}")
    print("- Capacités respectées: ", end="")
    for i in noeuds:
        if g[i].X > P_i[i]:
            print(f"\n  ERREUR nœud {i+1} ({g[i].X} > {P_i[i]})", end="")
    print("OK" if all(g[i].X <= P_i[i] for i in noeuds) else "")
    
    


SOLUTION OPTIMALE:
Coût total: 1275.00 €
Demande totale: 40 MW
Nœud 1: 0.0/15 MW (coût: 40 €/MWh)
Nœud 2: 15.0/20 MW (coût: 35 €/MWh)
Nœud 3: 0.0/10 MW (coût: 50 €/MWh)
Nœud 4: 25.0/25 MW (coût: 30 €/MWh)

Production totale: 40.0 MW
Vérification contraintes:
- Équilibre offre/demande: OK
- Capacités respectées: OK


Cela montre bien qu'on trouve la solution optimale. 

# Utilisation du développement binaire

Comme il y a le risque de dépasser la valeur de $P_i$ on rajoute une contrainte d'inégalité

### 2 noeuds

In [44]:
import numpy as np
import math
from dwave.samplers import SimulatedAnnealingSampler



# Paramètres du problème
P_i = [4, 2]       # Puissance max par noeud
c_i = [3, 8]        # Coût par unité de puissance
D_i = [3, 1]        # Demande par noeud
D_total = sum(D_i)       # 40 MW

# Coefficients de pénalité RENFORCÉS
lam= 10 


n_bits = [math.ceil(math.log2(p + 1)) for p in P_i]  # [4, 5, 4, 5]
total_vars = sum(n_bits)  

print(f"Nombre total de variables binaires: {total_vars}")

# Mapping des variables (i, bit) → index global
mapping = {}
index = 0
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        mapping[(i, j)] = index
        index += 1

# CONSTRUCTION DU QUBO OPTIMISÉ
Q = {}

# 1. Termes de coût (linéaires)
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        idx = mapping[(i, j)]
        weight = 2**j
        Q[(idx, idx)] = c_i[i] * weight

# Contrainte d'égalité (∑g_i = D_total) 
for idx1 in range(total_vars):
    for idx2 in range(idx1, total_vars):
        # Trouver les poids binaires
        a = next(2**j for (i,j), idx in mapping.items() if idx == idx1)
        b = a if idx1 == idx2 else next(2**j for (i,j), idx in mapping.items() if idx == idx2)
        
        coef = lam * a * b
        if idx1 == idx2:
            Q[(idx1, idx1)] = Q.get((idx1, idx1), 0) + coef - 2 * lam * D_total * a
        else:
            Q[(idx1, idx2)] = Q.get((idx1, idx2), 0) + 2 * coef

# Ajout du terme constant (nécessaire pour l'égalité)
constant_term = lam* D_total**2


sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample_qubo(Q, num_reads=1000, seed=1234)  # Seed pour reproductibilité
sample = sampleset.first.sample



g = [0] * len(P_i)
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        if sample[mapping[(i, j)]] == 1:
            g[i] += 2**j


total_production = sum(g)
total_cost = sum(c_i[i] * g[i] for i in range(len(g)))

print("\n RÉSULTATS QA:")
print(f"Production: {g} MW")
print(f"Production totale: {total_production} MW")
print(f"Demande totale: {D_total} MW")
print(f"Coût total: {total_cost} €")
print(f"Valeur du terme constant: {constant_term}")

print("\nContraintes de capacité:")
for i, (prod, p_max) in enumerate(zip(g, P_i)):
    status = "OK" if prod <= p_max else f"DÉPASSEMENT ({prod} > {p_max})"
    print(f"Nœud {i+1}: {prod}/{p_max} MW - {status}")

print("\nÉquilibre offre/demande:", "OK" if total_production == D_total else f"ERREUR ({total_production} ≠ {D_total})")

Nombre total de variables binaires: 5

 RÉSULTATS QA:
Production: [4, 0] MW
Production totale: 4 MW
Demande totale: 4 MW
Coût total: 12 €
Valeur du terme constant: 160

Contraintes de capacité:
Nœud 1: 4/4 MW - OK
Nœud 2: 0/2 MW - OK

Équilibre offre/demande: OK


### 4 noeuds

In [59]:
# Paramètres du problème
P_i = [15, 20, 10, 25]  
c_i = [40, 35, 50, 30]  
D_i = [12, 8, 5, 15] 
D_total = sum(D_i)       

lambda1 = 20000


n_bits = [math.ceil(math.log2(p + 1)) for p in P_i]  
total_vars = sum(n_bits) 

print(f"Nombre total de variables binaires: {total_vars}")

# Mapping des variables (i, bit) → index global
mapping = {}
index = 0
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        mapping[(i, j)] = index
        index += 1

# CONSTRUCTION DU QUBO OPTIMISÉ
Q = {}

#Termes de coût (linéaires)
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        idx = mapping[(i, j)]
        weight = 2**j
        Q[(idx, idx)] = c_i[i] * weight

#Contrainte d'égalité (∑g_i = D_total)
for idx1 in range(total_vars):
    for idx2 in range(idx1, total_vars):
        # Trouver les poids binaires
        a = next(2**j for (i,j), idx in mapping.items() if idx == idx1)
        b = a if idx1 == idx2 else next(2**j for (i,j), idx in mapping.items() if idx == idx2)
        
        coef = lambda1 * a * b
        if idx1 == idx2:
            Q[(idx1, idx1)] = Q.get((idx1, idx1), 0) + coef - 2 * lambda1 * D_total * a
        else:
            Q[(idx1, idx2)] = Q.get((idx1, idx2), 0) + 2 * coef

# Ajout du terme constant (nécessaire pour l'égalité)
constant_term = lambda1 * D_total**2


sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample_qubo(Q, num_reads=1000, seed=1234)  # Seed pour reproductibilité
sample = sampleset.first.sample

g = [0] * len(P_i)
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        if sample[mapping[(i, j)]] == 1:
            g[i] += 2**j

total_production = sum(g)
total_cost = sum(c_i[i] * g[i] for i in range(len(g)))

print("\n RÉSULTATS QA:")
print(f"Production: {g} MW")
print(f"Production totale: {total_production} MW")
print(f"Demande totale: {D_total} MW")
print(f"Coût total: {total_cost} €")
print(f"Valeur du terme constant: {constant_term}")

print("\nContraintes de capacité:")
for i, (prod, p_max) in enumerate(zip(g, P_i)):
    status = "OK" if prod <= p_max else f"DÉPASSEMENT ({prod} > {p_max})"
    print(f"Nœud {i+1}: {prod}/{p_max} MW - {status}")

print("\nÉquilibre offre/demande:", "OK" if total_production == D_total else f"ERREUR ({total_production} ≠ {D_total})")

Nombre total de variables binaires: 18

 RÉSULTATS QA:
Production: [4, 5, 0, 31] MW
Production totale: 40 MW
Demande totale: 40 MW
Coût total: 1265 €
Valeur du terme constant: 32000000

Contraintes de capacité:
Nœud 1: 4/15 MW - OK
Nœud 2: 5/20 MW - OK
Nœud 3: 0/10 MW - OK
Nœud 4: 31/25 MW - DÉPASSEMENT (31 > 25)

Équilibre offre/demande: OK


Avec la représentation binaire on a le risque du dépassement de la valeur maximale de $g_i$, pour cela on rajoute la contrainte d'inégalité dans la formulation QUBO.

In [69]:
import numpy as np
import math
from dwave.samplers import SimulatedAnnealingSampler


P_i = [15, 20, 10, 25]   # Capacités max
c_i = [40, 35, 50, 30]   # Coûts unitaires
D_i = [12, 8, 5, 15]     
D_total = sum(D_i)       # Demande totale

# Coefficients de pénalité
lambda1 = 20000  # Contrainte d'égalité (∑g_i = D_total)
lambda2 = 1000    # Contrainte de capacité (g_i ≤ P_i)


n_bits = [math.ceil(math.log2(p + 1)) for p in P_i]              # bits pour g_i
slack_bits = [math.ceil(math.log2(P_i[i] + 1)) for i in range(len(P_i))]  # bits slack

total_vars = sum(n_bits) + sum(slack_bits)
print(f"Nombre total de variables binaires: {total_vars}")

# Mapping des indices
mapping = {}
index = 0

# Bits g_i
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        mapping[("g", i, j)] = index
        index += 1

# Bits slack s_i
for i in range(len(P_i)):
    for j in range(slack_bits[i]):
        mapping[("s", i, j)] = index
        index += 1

# Construction du QUBO
Q = {}

# Terme de coût : c_i * g_i
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        idx = mapping[("g", i, j)]
        Q[(idx, idx)] = c_i[i] * (2**j)

# Contrainte d'égalité (∑g_i = D_total)
for i1 in range(len(P_i)):
    for j1 in range(n_bits[i1]):
        idx1 = mapping[("g", i1, j1)]
        a = 2**j1
        
        # Terme linéaire
        Q[(idx1, idx1)] = Q.get((idx1, idx1), 0) + lambda1 * (a**2 - 2 * D_total * a)
        
        # Termes croisés g_i-g_j
        for i2 in range(len(P_i)):
            for j2 in range(n_bits[i2]):
                idx2 = mapping[("g", i2, j2)]
                if idx1 < idx2:
                    b = 2**j2
                    Q[(idx1, idx2)] = Q.get((idx1, idx2), 0) + lambda1 * 2 * a * b

# Contrainte de capacité via slack : g_i + s_i = P_i
for i in range(len(P_i)):
    # Bits g_i
    for j1 in range(n_bits[i]):
        idx1 = mapping[("g", i, j1)]
        a = 2**j1
        Q[(idx1, idx1)] = Q.get((idx1, idx1), 0) + lambda2 * (a**2 - 2 * P_i[i] * a)
        
        # Croisés g_i-g_i
        for j2 in range(j1 + 1, n_bits[i]):
            idx2 = mapping[("g", i, j2)]
            b = 2**j2
            Q[(idx1, idx2)] = Q.get((idx1, idx2), 0) + lambda2 * 2 * a * b
        
        # Croisés g_i - slack
        for k in range(slack_bits[i]):
            idx_s = mapping[("s", i, k)]
            w = 2**k
            Q[(idx1, idx_s)] = Q.get((idx1, idx_s), 0) + lambda2 * 2 * a * w

    # Bits slack
    for j1 in range(slack_bits[i]):
        idx1 = mapping[("s", i, j1)]
        w1 = 2**j1
        Q[(idx1, idx1)] = Q.get((idx1, idx1), 0) + lambda2 * (w1**2 - 2 * P_i[i] * w1)
        
        # Croisés slack-slack
        for j2 in range(j1 + 1, slack_bits[i]):
            idx2 = mapping[("s", i, j2)]
            w2 = 2**j2
            Q[(idx1, idx2)] = Q.get((idx1, idx2), 0) + lambda2 * 2 * w1 * w2

# Résolution
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample_qubo(Q, num_reads=1000)
sample = sampleset.first.sample

# Reconstruction de g_i
g = [0] * len(P_i)
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        if sample[mapping[("g", i, j)]] == 1:
            g[i] += 2**j

# Reconstruction du slack
s = [0] * len(P_i)
for i in range(len(P_i)):
    for j in range(slack_bits[i]):
        if sample[mapping[("s", i, j)]] == 1:
            s[i] += 2**j

# Affichage des résultats
total_production = sum(g)
total_cost = sum(c_i[i] * g[i] for i in range(len(g)))

print("\nRÉSULTATS QA:")
print(f"Production: {g} MW")
print(f"Slack: {s} MW")
print(f"Production totale: {total_production} MW")
print(f"Demande totale: {D_total} MW")
print(f"Coût total: {total_cost} €")

print("\nContraintes de capacité:")
for i, (prod, p_max) in enumerate(zip(g, P_i)):
    status = "OK" if prod <= p_max else f"DÉPASSEMENT ({prod} > {p_max})"
    print(f"Nœud {i+1}: {prod}/{p_max} MW - {status}")

print("\nÉquilibre offre/demande:", "OK" if total_production == D_total else f"ERREUR ({total_production} ≠ {D_total})")


Nombre total de variables binaires: 36

RÉSULTATS QA:
Production: [0, 15, 0, 25] MW
Slack: [15, 5, 10, 0] MW
Production totale: 40 MW
Demande totale: 40 MW
Coût total: 1275 €

Contraintes de capacité:
Nœud 1: 0/15 MW - OK
Nœud 2: 15/20 MW - OK
Nœud 3: 0/10 MW - OK
Nœud 4: 25/25 MW - OK

Équilibre offre/demande: OK
